In [1]:
import pandas as pd
import string
import numpy as np
import nltk
from nltk.corpus import stopwords
import time

In [3]:
# nltk.download_shell()

In [6]:
train=pd.read_csv(r'C:\Users\Ismat\OneDrive\Рабочий стол\document\etc\lesson_course\projects\nlp__kaggle_competition\train.csv')

In [7]:
test=pd.read_csv(r'C:\Users\Ismat\OneDrive\Рабочий стол\document\etc\lesson_course\projects\nlp__kaggle_competition\test.csv')

In [12]:
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [9]:
test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [12]:
train.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [23]:
no_punc=[c for c in train['text']  if c not in string.punctuation]
no_punc=' '.join(no_punc)

In [38]:
clean_text=[word for word in no_punc.split() if word.lower() not in stopwords.words('english')]

In [10]:
def text_cleaner (text) :
    punctuation = [char for char in text if char not in string.punctuation]
    punctuation = ''.join (punctuation)
    return [word for word in punctuation.split() if word.lower() not in stopwords.words('english')]

In [16]:
%%time
train['text'].apply(text_cleaner)

CPU times: total: 11.9 s
Wall time: 12.3 s


0       [Deeds, Reason, earthquake, May, ALLAH, Forgiv...
1           [Forest, fire, near, La, Ronge, Sask, Canada]
2       [residents, asked, shelter, place, notified, o...
3       [13000, people, receive, wildfires, evacuation...
4       [got, sent, photo, Ruby, Alaska, smoke, wildfi...
                              ...                        
7608    [Two, giant, cranes, holding, bridge, collapse...
7609    [ariaahrary, TheTawniest, control, wild, fires...
7610    [M194, 0104, UTC5km, Volcano, Hawaii, httptcoz...
7611    [Police, investigating, ebike, collided, car, ...
7612    [Latest, Homes, Razed, Northern, California, W...
Name: text, Length: 7613, dtype: object

In [44]:
from sklearn.feature_extraction.text import CountVectorizer

In [46]:
%%time
transformer = CountVectorizer(analyzer=text_cleaner).fit(train['text'])

CPU times: total: 13.1 s
Wall time: 15.5 s


In [ ]:
len(transformer.vocabulary_)

26473

In [50]:
transformer_bow = transformer.transform( train['text'] )

In [ ]:
transformer_bow.shape

(7613, 26473)

In [ ]:
transformer_bow.nnz

75006

In [55]:
from sklearn.feature_extraction.text import TfidfTransformer

In [56]:
tf_idf_transformer = TfidfTransformer().fit(transformer_bow)

In [57]:
text_tfidf = tf_idf_transformer.transform(transformer_bow)

In [58]:
from sklearn.naive_bayes import MultinomialNB

In [59]:
disaster_definer_model = MultinomialNB().fit(text_tfidf,train['target'])

In [65]:
all_data_predictions = disaster_definer_model.predict(text_tfidf)

In [66]:
from sklearn.metrics import classification_report

print(classification_report(train['target'],all_data_predictions))

              precision    recall  f1-score   support

           0       0.88      0.98      0.93      4342
           1       0.97      0.83      0.89      3271

    accuracy                           0.91      7613
   macro avg       0.93      0.90      0.91      7613
weighted avg       0.92      0.91      0.91      7613



## splitting model with pipline train and test set for performance testing

In [72]:
from sklearn.model_selection import train_test_split

In [105]:
futures_train,futures_test,target_train,target_test = train_test_split(train['text'],train['target'],test_size=.2) 

In [74]:
from sklearn.pipeline import Pipeline

In [75]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_cleaner)),
    ('tfidf', TfidfTransformer()),
    ('classifier',MultinomialNB())
])

In [76]:
pipeline.fit(futures_train,target_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_cleaner at 0x000001A1720FA280>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [95]:
predictions_for_metric_evaluation = pipeline.predict(futures_test)

In [98]:
print(classification_report(target_test,predictions_for_metric_evaluation))

              precision    recall  f1-score   support

           0       0.76      0.91      0.83       861
           1       0.85      0.63      0.72       662

    accuracy                           0.79      1523
   macro avg       0.80      0.77      0.78      1523
weighted avg       0.80      0.79      0.78      1523



## for submission file

In [109]:
predictions_for_submission = pipeline.predict(test['text'])
len(predictions_for_submission)

3263

In [110]:
output = pd.DataFrame({'id': test.id, 'target': predictions_for_submission})
output

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,0
3259,10865,0
3260,10868,1
3261,10874,1


In [111]:
output.to_csv('NB.csv', index=False)